In [7]:
# Install packages
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from scipy.sparse import hstack
from sklearn.metrics import classification_report
import sklearn
import pickle
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import feature_selection

In [2]:
# Load data
df = pd.read_csv("mbti_preprocessed_features.csv", index_col= 0)

In [3]:
def create_train_test_split(X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42069)
    return X_train, X_test, Y_train, Y_test

In [9]:
feature = ['avg_word_count', 'avg_exclam_count', 'avg_fullstop_count', 'avg_count_of_hello', 'avg_count_of_hi', 'avg_count_of_extroverted_bigrams', 'avg_count_of_extroverted_stylistic_impressions',	'avg_count_of_interoverted_quantifiers',	'avg_count_of_introverted_first_person_singular_pronoun',	'avg_count_of_introverted_negations']

In [10]:
# Multiclass + Features - Unbalanced

# Determine X and Y
X = df[['preprocessed_posts', 'avg_word_count', 'avg_exclam_count', 'avg_fullstop_count', 'avg_count_of_hello', 'avg_count_of_hi', 'avg_count_of_extroverted_bigrams', 'avg_count_of_extroverted_stylistic_impressions',	'avg_count_of_interoverted_quantifiers',	'avg_count_of_introverted_first_person_singular_pronoun',	'avg_count_of_introverted_negations' ]] #Preprocessed_posts #Preprocessed_Posts + Features
Y = df.iloc[:, 2].values #EncodedType

# Create splits
X_train, X_test, Y_train, Y_test = create_train_test_split(X, Y)

## TF-IDF
vectorizer = TfidfVectorizer(min_df = 0.1, max_df=0.3, max_features=10000)

##Extract Vocabulary
corpus = X_train["preprocessed_posts"].values.astype(str)
vectorizer.fit(corpus)
X_training = vectorizer.transform(corpus)
dic_vocabulary = vectorizer.vocabulary_
# print("Training vocabulary size before dimension reduction: " + str(len(dic_vocabulary)))

# Feature Selection

##Reduce Dimensionality for sparse data with Chi-Quadrat
X_names = vectorizer.get_feature_names_out()
p_value_limit = 0.95
features = pd.DataFrame()
# print("Top Features for Each Class:")
for cat in np.unique(Y_train):
    chi2, p = feature_selection.chi2(X_training, Y_train == cat)
    features = features.append(pd.DataFrame(
        {"feature": X_names, "score": 1 - p, "y": cat}))
    features = features.sort_values(["y", "score"], ascending=[True, False])
    features = features[features["score"] > p_value_limit]
X_names = features["feature"].unique().tolist()

##Re-Fit vectorizer on corpus with new set of words and create new sparse matrix
vectorizer = TfidfVectorizer(vocabulary=X_names)
vectorizer.fit(corpus)

df_tfidfvect = vectorizer.transform(corpus)
df_tfidfvect = hstack([df_tfidfvect, X_train[feature]])
tfidf_tokens = vectorizer.get_feature_names()
df_tfidfvect = pd.DataFrame(data = df_tfidfvect.toarray(),columns = tfidf_tokens + feature)

# Normalise Features
scaler = MinMaxScaler()
df_tfidfvect[feature] = scaler.fit_transform(df_tfidfvect[feature])

corpus = X_test["preprocessed_posts"].values.astype(str)
df_tfidfvect_test = vectorizer.transform(corpus)
dic_vocabulary = vectorizer.vocabulary_

df_tfidfvect_test = hstack([df_tfidfvect_test, X_test[feature]])
tfidf_tokens_test = vectorizer.get_feature_names()
df_tfidfvect_test = pd.DataFrame(data = df_tfidfvect_test.toarray(),columns = tfidf_tokens_test + feature)

# Normalise Features
df_tfidfvect_test[feature] = scaler.transform(df_tfidfvect_test[feature])

# Train Model
dt = DecisionTreeClassifier()
dt = dt.fit(df_tfidfvect, Y_train)

# Prediction & Evaluation
prediction = dt.predict(df_tfidfvect_test)

types = df.iloc[:, 0].values
types = sorted(list(set(types)))

print("Classification Report:")
print(classification_report(Y_test, prediction, target_names=types))
print(sklearn.metrics.f1_score(Y_test, prediction, average ='micro'))

c:\Users\ricar\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\ricar\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Classification Report:
              precision    recall  f1-score   support

        ENFJ       0.34      0.38      0.36        55
        ENFP       0.36      0.37      0.37       212
        ENTJ       0.32      0.28      0.30        74
        ENTP       0.42      0.46      0.44       196
        ESFJ       0.00      0.00      0.00        11
        ESFP       0.00      0.00      0.00        14
        ESTJ       0.00      0.00      0.00        11
        ESTP       0.25      0.11      0.15        28
        INFJ       0.46      0.48      0.47       436
        INFP       0.51      0.49      0.50       545
        INTJ       0.48      0.41      0.44       365
        INTP       0.44      0.47      0.45       378
        ISFJ       0.42      0.33      0.37        58
        ISFP       0.15      0.18      0.16        73
        ISTJ       0.28      0.34      0.31        61
        ISTP       0.40      0.45      0.42        86

    accuracy                           0.42      2603
   

In [11]:
# Multiclass + Features - Balanced

# Determine X and Y
# Determine X and Y
X = df[['preprocessed_posts', 'avg_word_count', 'avg_exclam_count', 'avg_fullstop_count', 'avg_count_of_hello', 'avg_count_of_hi', 'avg_count_of_extroverted_bigrams', 'avg_count_of_extroverted_stylistic_impressions',	'avg_count_of_interoverted_quantifiers',	'avg_count_of_introverted_first_person_singular_pronoun',	'avg_count_of_introverted_negations' ]] #Preprocessed_posts #Preprocessed_Posts + Features
Y = df.iloc[:, 2].values #EncodedType

# Create splits
X_train, X_test, Y_train, Y_test = create_train_test_split(X, Y)

# balance data
sampler = RandomOverSampler()
X_train_balanced, Y_train_balanced = sampler.fit_resample(X_train, Y_train)

## TF-IDF
vectorizer = TfidfVectorizer(min_df = 0.1, max_df=0.3, max_features=10000)

##Extract Vocabulary
corpus = X_train_balanced["preprocessed_posts"].values.astype(str)
vectorizer.fit(corpus)
X_training = vectorizer.transform(corpus)
dic_vocabulary = vectorizer.vocabulary_
# print("Training vocabulary size before dimension reduction: " + str(len(dic_vocabulary)))

# Feature Selection

##Reduce Dimensionality for sparse data with Chi-Quadrat
X_names = vectorizer.get_feature_names_out()
p_value_limit = 0.95
features = pd.DataFrame()
# print("Top Features for Each Class:")
for cat in np.unique(Y_train_balanced):
    chi2, p = feature_selection.chi2(X_training, Y_train_balanced == cat)
    features = features.append(pd.DataFrame(
        {"feature": X_names, "score": 1 - p, "y": cat}))
    features = features.sort_values(["y", "score"], ascending=[True, False])
    features = features[features["score"] > p_value_limit]
X_names = features["feature"].unique().tolist()

##Re-Fit vectorizer on corpus with new set of words and create new sparse matrix
vectorizer = TfidfVectorizer(vocabulary=X_names)
vectorizer.fit(corpus)

df_tfidfvect = vectorizer.transform(corpus)
df_tfidfvect = hstack([df_tfidfvect, X_train_balanced[feature]])
tfidf_tokens = vectorizer.get_feature_names()
df_tfidfvect = pd.DataFrame(data = df_tfidfvect.toarray(),columns = tfidf_tokens + feature)

# Normalise Features
scaler = MinMaxScaler()
df_tfidfvect[feature] = scaler.fit_transform(df_tfidfvect[feature])

corpus = X_test["preprocessed_posts"].values.astype(str)
df_tfidfvect_test = vectorizer.transform(corpus)
dic_vocabulary = vectorizer.vocabulary_

df_tfidfvect_test = hstack([df_tfidfvect_test, X_test[feature]])
tfidf_tokens_test = vectorizer.get_feature_names()
df_tfidfvect_test = pd.DataFrame(data = df_tfidfvect_test.toarray(),columns = tfidf_tokens_test + feature)

# Normalise Features
df_tfidfvect_test[feature] = scaler.transform(df_tfidfvect_test[feature])

# Train Model
dt = DecisionTreeClassifier()
dt = dt.fit(df_tfidfvect, Y_train_balanced)

# Prediction & Evaluation
prediction = dt.predict(df_tfidfvect_test)

types = df.iloc[:, 0].values
types = sorted(list(set(types)))

print("Classification Report:")
print(classification_report(Y_test, prediction, target_names=types))
print(sklearn.metrics.f1_score(Y_test, prediction, average ='micro'))

c:\Users\ricar\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\ricar\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Classification Report:
              precision    recall  f1-score   support

        ENFJ       0.33      0.35      0.34        55
        ENFP       0.34      0.37      0.36       212
        ENTJ       0.37      0.26      0.30        74
        ENTP       0.37      0.39      0.38       196
        ESFJ       0.31      0.36      0.33        11
        ESFP       0.00      0.00      0.00        14
        ESTJ       0.19      0.27      0.22        11
        ESTP       0.13      0.11      0.12        28
        INFJ       0.39      0.39      0.39       436
        INFP       0.45      0.45      0.45       545
        INTJ       0.36      0.31      0.33       365
        INTP       0.35      0.38      0.37       378
        ISFJ       0.30      0.28      0.29        58
        ISFP       0.31      0.33      0.32        73
        ISTJ       0.23      0.20      0.21        61
        ISTP       0.29      0.37      0.32        86

    accuracy                           0.37      2603
   